In [153]:
import numpy as np
import scipy.stats as stats
import pandas as pd


class AssetDynamics:
    def __init__(self, s0, dt, mu, vol):
        self.s = s0
        self.dt = dt
        self.mu = mu
        self.vol = vol
        self.t = 0
        self.bid_spread = 50 / 10000
        self.ask_spread = 50 / 10000
    def evolve(self):
        self.s *= np.exp(self.mu * self.dt - 0.5*self.vol**2*self.dt + self.vol*self.dt**0.5*np.random.normal())
        self.t += self.dt
    def bid_price(self):
        return self.s *(1 + self.bid_spread)
        
    def ask_price(self):
        return self.s *(1 + self.ask_spread)
        

class Pnl:
    def __init__(self):
        self.spread = 0
        self.inventory = 0
        self.hdg = 0
    def total_pnl(self):
        return self.spread + self.inventory + self.hdg
    def __str__(self):
        return 'spread: {}, inventory: {}, hdg: {}'.format(self.spread, self.inventory, self.hdg)
        
class Inventory:
    def __init__(self):
        self.units = 0
    def __str__(self):
        return 'inventory: {}'.format(self.units)
    
class OBEntry:
    def __init__(self, price, size, agent):
        self.price = price
        self.size = size
        self.agent = agent
        
class RFQ:
    def __init__(self, sign, size):
        self.sign = sign
        self.size = size
        
class Transaction:
    def __init__(self, sn, time, agent, investor, sign, size, price, spread):
        self.sn = sn # serial number
        self.time = time
        self.agent = agent
        self.investor = investor
        self.sign = sign
        self.size = size
        self.price = price
        self.spread = spread
    
class MathingEngine:
    def __init__(self):
        self.LOB_bid = None
        self.LOB_ask = None
    def publish(self, obentry):
        
    
        
class Engine:
    def __init__(self):
        self.exchange = None
        self.investors = []
        self.agents = []
        self.transactions = []
    def set_exchange(self, exchange):
        self.exchange = exchange
    def add_investor(self, investor):
        self.investors.append(investor)
    def add_agent(self, agent):
        self.agents.append(agent)
        
    def transactions_df(self):
        fields = ['sn', 'time', 'agent', 'investor', 'sign', 'size', 'price', 'spread']
        #tmp = [{'sn': getattr(f, 'sn')} for f in self.transactions]
        df = pd.DataFrame([{fn: getattr(f, fn) for fn in fields} for f in self.transactions])
        df.rename(inplace=True, columns={'spread':'spread_bp'}) 
        df['spread_bp'] = df['spread_bp']*10000
        return df
        
    def evolve(self):
        self.exchange.evolve()
    
    def execute_match(self):
        # Assume agents quote infinite size. TODO: implement diff sizes.
        bo_sprds = [a.quote_spreads(self.exchange) for a in self.agents]
        offer_sprds, bid_sprds = zip(*bo_sprds)
        bb_idx = np.argmin(bid_sprds)
        bo_idx = np.argmin(offer_sprds)
        bb_spr, bb_agent = bid_sprds[bb_idx], self.agents[bb_idx]
        bo_spr, bo_agent = bid_sprds[bo_idx], self.agents[bo_idx]
        bb_price = self.exchange.s - bb_spr
        bo_price = self.exchange.s + bo_spr
        
        # Investors
        trans = []
        sn = len(self.transactions)
        for i in self.investors:
            trd = i.gentrade()
            if trd:
                if trd.sign > 0: #Investor buys
                    tmp = Transaction(sn=sn, time=self.exchange.t, agent=bo_agent, investor=i, sign=trd.sign, size=trd.size, price=bo_price, spread=bo_spr)
                elif trd.sign < 0: #Investor sells
                    tmp = Transaction(sn=sn, time=self.exchange.t, agent=bo_agent, investor=i, sign=trd.sign, size=trd.size, price=bb_price, spread=bb_spr)
                else:
                    raise Exception()
                trans.append(tmp)
                sn += 1
                
        # execute (pnl)
        for t in trans:
            t.agent.inventory.units += -t.sign * t.size
            t.agent.pnl.spread += t.spread * t.size
            
        self.transactions.extend(trans)
                
            
# =======================================
# ==========  INVESTORS =================
# =======================================
class Investor:
    id_counter = 0
    def __init__(self):
        self.uid = Investor.id_counter
        Investor.id_counter += 1
        arrival_intensity = 1
        self.arrival_dist = stats.poisson(arrival_intensity)
        buy_prob = 0.5
        self.dir_dist = stats.bernoulli(buy_prob)
        size_rate = 1.0
        self.size_dist = stats.expon(size_rate)
    def gentrade(self):
        if self.arrival_dist.rvs():
            sign = 2*self.dir_dist.rvs() - 1
            size = self.size_dist.rvs()
            return RFQ(sign, size)
        else:
            return None
    def __str__(self):
        return '{} {}'.format(type(self).__name__, self.uid)
        
# =======================================
# ==========  AGENTS ====================
# =======================================
        
class Agent:
    def __init__(self):
        self.inventory = Inventory()
        self.pnl = Pnl()
    def quote_spreads():
        raise NotImplementedError()
        
class RandomAgent(Agent):
    id_counter = 0
    def __init__(self, eps_min, eps_max):
        Agent.__init__(self)
        self.uid = RandomAgent.id_counter
        RandomAgent.id_counter += 1
        self.eps_buy_dist = stats.uniform(loc=eps_min, scale=eps_max - eps_min)
        self.eps_sell_dist = stats.uniform(loc=eps_min, scale=eps_max - eps_min)
        self.hdgfrac_dist = stats.uniform(loc=0, scale=1)
    def quote_spreads(self, exchange):
        # Agent's streamed bid
        si_bid = exchange.bid_spread * (1 + self.eps_buy_dist.rvs())
        
        # Agent's streamed bid
        si_ask = exchange.ask_spread * (1 + self.eps_sell_dist.rvs())
        return (si_bid, si_ask)
    
    def __str__(self):
        return type(self).__name__ + ' ' + str(self.uid)
        
class PersistentAgent(Agent):
    id_counter = 0
    def __init__(self, eps_buy, eps_sell, hdgfrac):
        Agent.__init__(self)
        self.uid = RandomAgent.id_counter
        RandomAgent.id_counter += 1
        self.eps_buy = eps_buy
        self.eps_sell = eps_sell
        self.hdgfrac = hdgfrac
    def quote_spreads(self, exchange):
        si_bid = exchange.bid_spread * (1 + self.eps_buy)
        si_ask = exchange.ask_spread * (1 + self.eps_sell)
        return (si_bid, si_ask)
    
    def __str__(self):
        return type(self).__name__ + ' ' + str(self.uid)
        
        
# =======================================
# ============  INIT ====================
# =======================================
        
invtrs = [Investor()]
ag1 = PersistentAgent(eps_buy=0.0, eps_sell=0.0, hdgfrac=0)
ag2 = RandomAgent(eps_min=-0.5, eps_max=0.5)
ag3 = RandomAgent(eps_min=-1.0, eps_max=1.0)
agnts = [ag1, ag2, ag3]

eng = Engine()
exchange = AssetDynamics(s0=100, dt=1, mu=0, vol=0.000000001)
eng.set_exchange(exchange)
for i in invtrs:
    eng.add_investor(i)
for a in agnts:
    eng.add_agent(a)

matchingEngine = MatchingEngine()
# =======================================
# =============  RUN ====================
# =======================================
for i in range(1, 20):
    # 1. Each Agent Publishes prices
    for a in agnts:
        matchingEngine.publish(a, a.quote_spreads())
    
    eng.execute_match()
    eng.evolve()

    
# =======================================
# ============  RESULTS =================
# =======================================
for a in agnts:
    print('==========')
    print(type(a).__name__)
    print(a.inventory)
    print(a.pnl)

print(' ========================')
print(' ===== TRANSACTIONS =====')
print(eng.transactions_df().round({'price': 4, 'size':2, 'spread_bp':2}))

PersistentAgent
inventory: 5.657614359102726
spread: 0.03806131680139782, inventory: 0, hdg: 0
RandomAgent
inventory: 0.7945078518369493
spread: 0.028126796106865175, inventory: 0, hdg: 0
RandomAgent
inventory: -0.7502944737271258
spread: 0.07812003541162497, inventory: 0, hdg: 0
 ===== TRANSACTIONS =====
                agent    investor     price  sign  size  sn  spread_bp  time
0   PersistentAgent 0  Investor 0   99.9967    -1  1.11   0      33.14     0
1       RandomAgent 2  Investor 0  100.0035     1  1.34   1      34.65     1
2       RandomAgent 2  Investor 0  100.0028     1  1.13   2      27.68     2
3       RandomAgent 2  Investor 0  100.0088     1  1.51   3      87.93     3
4       RandomAgent 1  Investor 0   99.9964    -1  3.09   4      36.30     4
5       RandomAgent 2  Investor 0   99.9961    -1  3.87   5      39.36     6
6       RandomAgent 1  Investor 0   99.9977    -1  1.24   6      22.91     7
7       RandomAgent 2  Investor 0  100.0051     1  3.80   7      50.57     8
